## Import Statements

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# general imports
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.datasets import make_spd_matrix
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from matplotlib import pyplot

In [ ]:
# import an optimizer
from scipy.optimize import minimize

In [ ]:
# the following is very important to use GridSearchCV to tune the hyperparameters and be more compliant with sklearn
from sklearn.base import BaseEstimator, RegressorMixin

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# import numba and to speed up processing time
from numba import jit

## Simulate Data

In [ ]:
n = 200
p = 1200

In [ ]:
beta = 0

In [ ]:
# this is our ground truth
beta = np.concatenate((([1]*7), ([0]*25), ([0.25]*5), ([0]*50), ([0.7]*15), ([0]*1098)))

In [ ]:
# Note: What we want to detect is the position of the actual information or "signal"
pos = np.where(beta!=0)

In [ ]:
# Shows you the indices where beta is different from zero
pos

(array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101]),)

In [ ]:
np.array(pos).shape

(1, 27)

Only 27 important variables/non-zero coefficients

In [ ]:
# We need toeplitz like toeplitz([1,0.8,0.8**2,0.8**3,0.8**4,...,0.8**1199)]), to incorporate our correlation structure
vctr = []
for i in range(p):
  vctr.append(0.8**i)

In [ ]:
vctr

[1.0,
 0.8,
 0.6400000000000001,
 0.5120000000000001,
 0.4096000000000001,
 0.3276800000000001,
 0.2621440000000001,
 0.20971520000000007,
 0.1677721600000001,
 0.13421772800000006,
 0.10737418240000006,
 0.08589934592000005,
 0.06871947673600004,
 0.054975581388800036,
 0.043980465111040035,
 0.03518437208883203,
 0.028147497671065624,
 0.022517998136852502,
 0.018014398509482003,
 0.014411518807585602,
 0.011529215046068483,
 0.009223372036854787,
 0.00737869762948383,
 0.005902958103587064,
 0.004722366482869652,
 0.0037778931862957215,
 0.0030223145490365774,
 0.002417851639229262,
 0.0019342813113834097,
 0.0015474250491067279,
 0.0012379400392853823,
 0.0009903520314283058,
 0.0007922816251426448,
 0.0006338253001141158,
 0.0005070602400912927,
 0.00040564819207303417,
 0.00032451855365842736,
 0.00025961484292674194,
 0.00020769187434139353,
 0.00016615349947311485,
 0.00013292279957849188,
 0.00010633823966279351,
 8.507059173023481e-05,
 6.805647338418786e-05,
 5.4445178707350

In [ ]:
mu = [0]*p
sigma = 3.5
# Generate the random samples.
np.random.seed(123)
x = np.random.multivariate_normal(mu, toeplitz(vctr), size=n) # This is where we generate some fake data
y = np.matmul(x,beta) + sigma*np.random.normal(0,1,n)

In [ ]:
x

array([[ 0.7674391 ,  0.34116086, -0.6231329 , ...,  0.45949521,
        -0.78532038, -0.31836898],
       [ 0.87036725,  1.37071935,  1.25179402, ..., -0.44029845,
         0.66017914,  1.56056631],
       [-0.61247227,  0.11132874, -1.07101579, ..., -0.36963318,
         0.60716062, -0.12975609],
       ...,
       [-0.2092827 , -0.33111593, -0.73910767, ...,  0.84465008,
         0.83112518,  0.82189189],
       [ 2.64253328,  2.83656568,  1.72663917, ..., -2.98316143,
        -2.41105897, -2.64189953],
       [-0.43124027, -0.28914122, -0.68169266, ...,  0.48539612,
         0.32552655, -0.77697803]])

In [ ]:
x.shape

(200, 1200)

In [ ]:
y

array([-25.10551989,   8.03463436,  -7.96281493,   5.21434154,
        -4.14046976,  -4.79249863,   8.88817664,   1.57656237,
        -2.79026059,   6.91599671, -27.18721323,  23.66819477,
        -3.67123745,   2.90373348,   3.16303279,   3.75313185,
        14.50774285,  -2.92310987,  -3.05305551,   1.70936055,
        -8.60997758,  -1.95161352,   3.26329159, -15.4768899 ,
         5.26191791,  18.5829722 , -10.25654568,   0.66591799,
        -2.06895739,   9.35398165, -13.46182646,  -6.42117856,
        -4.22372637,  -6.0364587 ,  -2.5031809 , -25.73902325,
        -7.60374873,  -5.98982352, -13.66708144,  -0.53020143,
        -7.87440299, -23.27677577,  12.93668924,  -8.66820021,
        -2.80100219, -16.2908329 ,  -3.90247275,  -2.17318319,
         0.53738138,  -4.60952518,   2.29557674,  -5.85836837,
        -1.68373837, -11.83894279,  17.90857593,   2.8944184 ,
         9.49527506,  -8.98521124, -13.10449558,  -1.07272244,
         6.84258729,  17.56439614,   4.9903312 ,  -2.24

In [ ]:
y.shape

(200,)

In [ ]:
np.mean(x[:,8])

-0.013593603942792874

Making 100 datasets

In [ ]:
X = []
Y = []
for i in range(0,100):
  xnew = np.random.multivariate_normal(mu, toeplitz(vctr), size=n)
  X.append(xnew)
  Y.append(np.matmul(xnew,beta) + sigma*np.random.normal(0,1,n))

## Square Root Lasso

In [ ]:
class SQRTLasso(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.01):
        self.alpha = alpha
    def fit(self, x, y):
        alpha=self.alpha
        def f_obj(x,y,beta,alpha):
          n =len(x)
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.sqrt(1/n*np.sum((y-x.dot(beta))**2)) + alpha*np.sum(np.abs(beta))
          return output
        def f_grad(x,y,beta,alpha):
          n=x.shape[0]
          p=x.shape[1]
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.array((-1/np.sqrt(n))*np.transpose(x).dot(y-x.dot(beta))/np.sqrt(np.sum((y-x.dot(beta))**2))+alpha*np.sign(beta)).flatten()
          return output
        def objective(beta):
          return(f_obj(x,y,beta,alpha))
        def gradient(beta):
          return(f_grad(x,y,beta,alpha))
        
        beta0 = np.ones((x.shape[1],1))
        output = minimize(objective, beta0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        beta = output.x
        self.coef_ = beta
        
    def predict(self, x):
        return x.dot(self.coef_)

In [ ]:
model = SQRTLasso(alpha = 0.1)

In [ ]:
%%time
model.fit(x,y.reshape(-1,1))

CPU times: user 3.58 s, sys: 2.97 s, total: 6.56 s
Wall time: 6.05 s


In [ ]:
model.coef_

array([ 1.22459961e+00,  1.52938092e+00,  7.80717575e-01, ...,
       -1.84326301e-07,  8.60941267e-06,  7.35081035e-07])

In [ ]:
beta_hat = model.coef_
print(beta_hat)

[ 1.22459961e+00  1.52938092e+00  7.80717575e-01 ... -1.84326301e-07
  8.60941267e-06  7.35081035e-07]


In [ ]:
beta_hat[abs(beta_hat)<0.05] = 0
print(beta_hat)

[1.22459961 1.52938092 0.78071758 ... 0.         0.         0.        ]


In [ ]:
# Detect the position of the estimated non-zero beta coefficients/significant variables
pos_sqrtlasso = np.where(beta_hat!=0)
print(pos_sqrtlasso)

(array([   0,    1,    2,    3,    4,    5,    6,    7,   18,   29,   34,
         35,   58,   64,   86,   87,   88,   89,   90,   91,   92,   93,
         94,   95,   96,   97,   98,   99,  100,  101,  108,  138,  143,
        156,  157,  167,  168,  169,  180,  184,  192,  226,  258,  265,
        291,  297,  300,  301,  356,  373,  382,  391,  401,  413,  459,
        460,  467,  469,  475,  480,  482,  538,  572,  575,  588,  634,
        660,  666,  667,  684,  691,  695,  717,  718,  733,  734,  744,
        745,  758,  770,  779,  785,  796,  805,  807,  824,  825,  858,
        860,  881,  897,  904,  912,  931,  984,  993, 1010, 1018, 1030,
       1037, 1068, 1070, 1094, 1128, 1167, 1196]),)


In [ ]:
print(np.array(pos_sqrtlasso).shape[1])

106


In [ ]:
# Check how many of these important variables were actually important
print(np.intersect1d(pos,pos_sqrtlasso))
print(np.intersect1d(pos,pos_sqrtlasso).shape)
# We were able to reconstruct all 27 ground truths

[  0   1   2   3   4   5   6  34  35  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101]
(24,)


Compute the L2 distance between the ground truth and your solution

In [ ]:
np.linalg.norm(model.coef_-beta,ord=2)

2.125068916576415

## SCAD

In [ ]:
class SCADRegression(BaseEstimator, RegressorMixin):
    def __init__(self, a=2,lam=1):
        self.a, self.lam = a, lam
  
    def fit(self, x, y):
        a = self.a
        lam   = self.lam

        def scad_penalty(beta_hat, lambda_val, a_val):
          is_linear = (np.abs(beta_hat) <= lambda_val)
          is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
          is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
          linear_part = lambda_val * np.abs(beta_hat) * is_linear
          quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
          constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
          return linear_part + quadratic_part + constant_part

        def scad_derivative(beta_hat, lambda_val, a_val):
          return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

        
        def scad(beta):
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          n = len(y)
          return 1/n*np.sum((y-x.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))

         
        def dscad(beta):
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          n = len(y)
          output = -2/n*np.transpose(x).dot(y-x.dot(beta))+scad_derivative(beta,lam,a)
          return output.flatten()
        
        p = x.shape[1]
        beta0 = np.zeros(p)
        output = minimize(scad, beta0, method='L-BFGS-B', jac=dscad,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 50,'disp': False})
        beta = output.x
        self.coef_ = beta
        
    def predict(self, x):
        return x.dot(self.coef_)

In [ ]:
model = SCADRegression(a = 0.1, lam=0.1)

In [ ]:
%%time
model.fit(x,y.reshape(-1,1))

CPU times: user 52.6 ms, sys: 62.4 ms, total: 115 ms
Wall time: 133 ms


In [ ]:
beta_hat = model.coef_

In [ ]:
beta_hat

array([ 0.6721517 ,  0.78603033,  0.63894403, ..., -0.10034327,
       -0.13424441, -0.11910554])

In [ ]:
beta_hat[abs(beta_hat)<0.05] = 0
print(beta_hat)

[ 0.6721517   0.78603033  0.63894403 ... -0.10034327 -0.13424441
 -0.11910554]


In [ ]:
pos_SCAD = np.where(beta_hat!=0)

In [ ]:
pos_SCAD

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,   12,   14,
          17,   18,   19,   20,   22,   23,   24,   25,   26,   28,   29,
          31,   32,   33,   34,   35,   36,   37,   38,   39,   40,   41,
          42,   43,   45,   49,   50,   58,   62,   63,   64,   65,   67,
          68,   75,   76,   79,   80,   81,   82,   85,   86,   87,   88,
          89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
         100,  101,  102,  103,  104,  112,  113,  114,  115,  117,  118,
         119,  120,  123,  124,  125,  127,  128,  129,  132,  133,  134,
         135,  136,  137,  139,  141,  143,  144,  148,  151,  152,  155,
         156,  157,  159,  164,  165,  167,  168,  170,  171,  173,  174,
         175,  179,  180,  181,  184,  189,  191,  192,  193,  194,  196,
         199,  200,  201,  202,  206,  207,  209,  210,  212,  213,  214,
         215,  216,  217,  219,  221,  222,  223,  224,  225,  226,  227,
         228,  230,  231,  232,  233, 

In [ ]:
np.array(pos_SCAD).shape[1]

735

We identified 735 important variables

In [ ]:
np.intersect1d(pos,pos_SCAD)

array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101])

In [ ]:
np.intersect1d(pos,pos_SCAD).shape

(27,)

We found all 27 non-zero coefficients

In [ ]:
np.linalg.norm(model.coef_-beta,ord=2)

3.7775795440964965

## Lasso Regression

In [ ]:
# Apply Lasso and check how many important variables are recovered
model = Lasso(alpha=0.1, fit_intercept=False, max_iter=5000) # no intercept because the mean is 0

In [ ]:
model.fit(x,y)

Lasso(alpha=0.1, fit_intercept=False, max_iter=5000)

In [ ]:
beta_hat = model.coef_

In [ ]:
beta_hat

array([ 0.87672602,  2.60465118,  0.16529373, ..., -0.        ,
       -0.        , -0.        ])

In [ ]:
beta_hat.shape

(1200,)

In [ ]:
pos_lasso = np.where(beta_hat!=0)

In [ ]:
pos_lasso

(array([   0,    1,    2,    3,    4,    5,    6,    7,   18,   29,   32,
          35,   39,   58,   63,   64,   86,   87,   88,   89,   90,   92,
          94,   95,   96,   97,   99,  100,  101,  108,  133,  138,  143,
         151,  156,  163,  164,  168,  180,  184,  191,  192,  200,  243,
         249,  250,  258,  265,  270,  274,  285,  291,  297,  300,  301,
         314,  356,  367,  374,  382,  391,  401,  413,  459,  469,  475,
         480,  481,  482,  498,  503,  504,  515,  518,  538,  572,  575,
         582,  586,  588,  592,  630,  634,  646,  652,  660,  666,  672,
         684,  695,  717,  723,  724,  733,  734,  745,  758,  769,  770,
         779,  785,  805,  807,  824,  858,  860,  871,  876,  881,  897,
         904,  912,  920,  934,  947,  955,  973,  984,  993, 1003, 1010,
        1012, 1018, 1025, 1026, 1031, 1033, 1037, 1054, 1068, 1093, 1094,
        1134, 1140, 1141, 1142, 1146, 1147, 1159, 1166, 1167, 1187, 1194,
        1196]),)

In [ ]:
np.array(pos_lasso).shape[1]

144

144 important variables/non-zero coefficients identified

In [ ]:
np.intersect1d(pos,pos_lasso)

array([  0,   1,   2,   3,   4,   5,   6,  32,  35,  87,  88,  89,  90,
        92,  94,  95,  96,  97,  99, 100, 101])

These are the non-zero coefficients/important variables Lasso correctly identified

In [ ]:
np.intersect1d(pos,pos_lasso).shape

(21,)

We were able to reconstruct 21 ground truths each time

## Ridge Regression

In [ ]:
model = Ridge(alpha=0.1, fit_intercept=False)

In [ ]:
model.fit(x,y)

Ridge(alpha=0.1, fit_intercept=False)

In [ ]:
beta_hat = model.coef_

In [ ]:
beta_hat

array([ 0.73288445,  0.81692647,  0.69790345, ..., -0.07933362,
       -0.05808182, -0.04977748])

In [ ]:
beta_hat.shape

(1200,)

In [ ]:
beta_hat[abs(beta_hat)<0.05] = 0

In [ ]:
beta_hat

array([ 0.73288445,  0.81692647,  0.69790345, ..., -0.07933362,
       -0.05808182,  0.        ])

In [ ]:
pos_ridge = np.where(beta_hat!=0)

In [ ]:
pos_ridge

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,   11,   12,
          13,   17,   18,   28,   29,   31,   32,   33,   34,   35,   36,
          37,   39,   42,   45,   46,   53,   54,   58,   59,   63,   64,
          65,   66,   73,   74,   75,   76,   77,   78,   81,   85,   86,
          87,   88,   89,   90,   91,   92,   93,   94,   95,   96,   97,
          98,   99,  100,  101,  102,  103,  104,  108,  111,  113,  115,
         116,  117,  120,  125,  126,  128,  130,  132,  133,  134,  135,
         138,  139,  140,  142,  143,  144,  145,  147,  149,  150,  151,
         152,  154,  155,  156,  157,  158,  159,  166,  167,  168,  169,
         174,  175,  177,  178,  180,  181,  184,  191,  192,  193,  194,
         196,  199,  200,  204,  206,  209,  210,  212,  213,  214,  215,
         216,  217,  219,  221,  222,  223,  224,  225,  226,  227,  231,
         232,  234,  235,  236,  237,  239,  240,  242,  243,  244,  245,
         247,  249,  252,  256,  257, 

In [ ]:
np.array(pos_ridge).shape[1]

627

We identified 627 important variables/non-zero coefficients.

In [ ]:
np.intersect1d(pos,pos_ridge)

array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101])

In [ ]:
np.intersect1d(pos,pos_ridge).shape

(27,)

We were able to reconstruct all 27 ground truths

## Elastic Net

In [ ]:
model = ElasticNet(alpha=0.1, fit_intercept=False)

In [ ]:
model.fit(x,y)

ElasticNet(alpha=0.1, fit_intercept=False)

In [ ]:
beta_hat = model.coef_

In [ ]:
beta_hat

array([ 1.12643364,  1.53685203,  0.72808469, ..., -0.        ,
       -0.        , -0.        ])

In [ ]:
beta_hat.shape

(1200,)

In [ ]:
pos_elasticnet = np.where(beta_hat!=0)

In [ ]:
pos_elasticnet

(array([   0,    1,    2,    3,    4,    5,    6,    7,   18,   28,   29,
          32,   34,   35,   46,   48,   58,   63,   64,   85,   86,   87,
          88,   89,   90,   91,   92,   93,   94,   95,   96,   97,   98,
          99,  100,  101,  108,  133,  135,  138,  143,  147,  151,  156,
         157,  163,  166,  167,  168,  169,  175,  180,  184,  191,  192,
         200,  217,  221,  226,  235,  248,  249,  250,  257,  258,  261,
         265,  270,  274,  276,  285,  291,  292,  297,  300,  301,  307,
         314,  315,  328,  356,  361,  362,  367,  374,  382,  391,  401,
         402,  413,  419,  423,  433,  459,  460,  467,  468,  469,  475,
         480,  481,  482,  490,  495,  497,  502,  504,  515,  518,  524,
         534,  538,  539,  557,  572,  573,  575,  581,  582,  586,  588,
         592,  602,  607,  622,  623,  627,  630,  631,  632,  634,  652,
         660,  662,  666,  684,  695,  715,  716,  717,  718,  721,  723,
         724,  731,  732,  734,  735, 

In [ ]:
np.array(pos_elasticnet).shape[1]

238

We identified 238 important variables/non-zero coefficients.

In [ ]:
np.intersect1d(pos,pos_elasticnet)

array([  0,   1,   2,   3,   4,   5,   6,  32,  34,  35,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101])

In [ ]:
np.intersect1d(pos,pos_elasticnet).shape

(25,)

We were able to reconstruct 25 ground truths

## RMSE Calculations

# Compare signficant variable counts/overlaps next

In [ ]:
%%time
model = Lasso(fit_intercept=False,max_iter=2500)
params = [{'alpha':np.linspace(0.001,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y)
print(gs_results.best_params_)
print('The root mean square error is: ', np.sqrt(np.abs(gs_results.best_score_)))

{'alpha': 0.16410204081632654}
The root mean square error is:  3.9358439757006853
CPU times: user 16.4 s, sys: 13.8 s, total: 30.1 s
Wall time: 18.1 s


In [ ]:
%%time
model = Lasso(fit_intercept=False,max_iter=2500)
params = [{'alpha':np.linspace(0.001,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y)
print(gs_results.best_params_)
print('The mean square error is: ', np.abs(gs_results.best_score_))

{'alpha': 0.16410204081632654}
The mean square error is:  15.490867801059377
CPU times: user 16.1 s, sys: 15.1 s, total: 31.2 s
Wall time: 17.5 s


In [ ]:
model = Lasso(alpha=0.16410204081632654, fit_intercept=False, max_iter=2500)

In [ ]:
model.fit(x,y)

Lasso(alpha=0.16410204081632654, fit_intercept=False, max_iter=2500)

In [ ]:
# Compute the L2 distance
np.linalg.norm(model.coef_-beta,ord=2)

3.8079724911118786

In [111]:
true_coeffs = []
L2Distance = []
RMSE = []
model = Lasso(alpha=0.16410204081632654, fit_intercept=False, max_iter=2500)
for i in range(0,100):
  model.fit(X[i], Y[i])
  true_coeffs.append(np.intersect1d(pos,np.where(model.coef_!=0)).shape[0])
  L2Distance.append(np.linalg.norm(model.coef_-beta,ord=2))
  RMSE.append(np.sqrt(MSE(Y[i],model.predict(X[i]))))

In [112]:
np.mean(true_coeffs)

20.88

In [113]:
np.mean(L2Distance)

3.1569210395048355

In [114]:
np.mean(RMSE)

1.7096307394995396

-----

In [ ]:
%%time
model = ElasticNet(fit_intercept=False, max_iter=500000)
params = [{'alpha':np.linspace(0.001,1,num=50),'l1_ratio':np.linspace(0.5,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y)
print(gs_results.best_params_)
print('The root mean square error is: ', np.sqrt(np.abs(gs_results.best_score_)))

{'alpha': 0.041775510204081635, 'l1_ratio': 0.6020408163265306}
The root mean square error is:  3.900448918082395
CPU times: user 11min 33s, sys: 9min 24s, total: 20min 57s
Wall time: 12min 38s


In [ ]:
%%time
model = ElasticNet(fit_intercept=False, max_iter=500000)
params = [{'alpha':np.linspace(0.001,1,num=50),'l1_ratio':np.linspace(0.5,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y)
print(gs_results.best_params_)
print('The mean square error is: ', np.abs(gs_results.best_score_))

{'alpha': 0.041775510204081635, 'l1_ratio': 0.6020408163265306}
The mean square error is:  15.213501762570123
CPU times: user 11min 9s, sys: 9min 37s, total: 20min 46s
Wall time: 11min 46s


In [76]:
model = ElasticNet(alpha=0.041775510204081635, l1_ratio=0.6020408163265306, fit_intercept=False, max_iter=500000)

In [77]:
model.fit(x,y)

ElasticNet(alpha=0.041775510204081635, fit_intercept=False,
           l1_ratio=0.6020408163265306, max_iter=500000)

In [78]:
# Compute the L2 distance
np.linalg.norm(model.coef_-beta,ord=2)

3.1210967781370713

In [115]:
true_coeffs = []
L2Distance = []
RMSE = []
model = ElasticNet(alpha=0.041775510204081635, l1_ratio=0.6020408163265306, fit_intercept=False, max_iter=500000)
for i in range(0,100):
  model.fit(X[i], Y[i])
  true_coeffs.append(np.intersect1d(pos,np.where(model.coef_!=0)).shape[0])
  L2Distance.append(np.linalg.norm(model.coef_-beta,ord=2))
  RMSE.append(np.sqrt(MSE(Y[i],model.predict(X[i]))))

In [116]:
np.mean(true_coeffs)

24.25

In [117]:
np.mean(L2Distance)

3.1935896740847407

In [118]:
np.mean(RMSE)

0.39124115023506123

----

In [ ]:
%%time
model = Ridge(fit_intercept=False,max_iter=1000)
params = [{'alpha':np.linspace(0.001,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y)
print(gs_results.best_params_)
print('The root mean square error is: ', np.sqrt(np.abs(gs_results.best_score_)))

{'alpha': 0.001}
The root mean square error is:  5.903268505036481
CPU times: user 5.15 s, sys: 5.91 s, total: 11.1 s
Wall time: 5.69 s


In [ ]:
%%time
model = Ridge(fit_intercept=False,max_iter=1000)
params = [{'alpha':np.linspace(0.001,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y)
print(gs_results.best_params_)
print('The mean square error is: ', np.abs(gs_results.best_score_))

{'alpha': 0.001}
The mean square error is:  34.84857904255565
CPU times: user 5.19 s, sys: 6.03 s, total: 11.2 s
Wall time: 5.78 s


In [79]:
model = Ridge(alpha=0.001, fit_intercept=False, max_iter=1000)

In [80]:
model.fit(x,y)

Ridge(alpha=0.001, fit_intercept=False, max_iter=1000)

In [81]:
# Compute the L2 distance
np.linalg.norm(model.coef_-beta,ord=2)

3.0028049415597535

In [119]:
true_coeffs = []
L2Distance = []
RMSE = []
model = Ridge(alpha=0.001, fit_intercept=False, max_iter=1000)
for i in range(0,100):
  model.fit(X[i], Y[i])
  true_coeffs.append(np.intersect1d(pos,np.where(model.coef_!=0)).shape[0])
  L2Distance.append(np.linalg.norm(model.coef_-beta,ord=2))
  RMSE.append(np.sqrt(MSE(Y[i],model.predict(X[i]))))

In [120]:
np.mean(true_coeffs)

27.0

In [121]:
np.mean(L2Distance)

3.149950533355907

In [122]:
np.mean(RMSE)

1.0124266845880489e-05

-----

#### SCAD

In [84]:
%%time
model = SCADRegression()
params = [{'a':np.linspace(0.001,1,num=50), 'lam':np.linspace(0.001,1,num=100)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y.reshape(-1,1))
print(gs_results.best_params_)
print('The root mean square error is: ', np.sqrt(np.abs(gs_results.best_score_)))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel i

{'a': 0.16410204081632654, 'lam': 0.001}
The root mean square error is:  5.920119991188657
CPU times: user 38min 10s, sys: 16min 24s, total: 54min 34s
Wall time: 27min 57s


In [85]:
%%time
model = SCADRegression()
params = [{'a':np.linspace(0.001,1,num=50), 'lam':np.linspace(0.001,1,num=100)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y.reshape(-1,1))
print(gs_results.best_params_)
print('The mean square error is: ', np.abs(gs_results.best_score_))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel i

{'a': 0.16410204081632654, 'lam': 0.001}
The mean square error is:  35.04782071007158
CPU times: user 38min 40s, sys: 16min 49s, total: 55min 29s
Wall time: 28min 27s


In [88]:
model = SCADRegression(a=0.16410204081632654, lam=0.001)

In [89]:
model.fit(x,y.reshape(-1,1))

In [90]:
# Compute the L2 distance
np.linalg.norm(model.coef_-beta,ord=2)

3.018786805585877

In [124]:
true_coeffs = []
L2Distance = []
RMSE = []
model = SCADRegression(a=0.16410204081632654, lam=0.001)
for i in range(0,100):
  model.fit(X[i], Y[i].reshape(-1,1))
  true_coeffs.append(np.intersect1d(pos,np.where(model.coef_!=0)).shape[0])
  L2Distance.append(np.linalg.norm(model.coef_-beta,ord=2))
  RMSE.append(np.sqrt(MSE(Y[i],model.predict(X[i]))))

In [125]:
np.mean(true_coeffs)

27.0

In [126]:
np.mean(L2Distance)

3.1633639693494104

In [127]:
np.mean(RMSE)

0.006658972592191754

-----

#### Square Root Lasso

In [91]:
%%time
model = SQRTLasso()
params = [{'alpha':np.linspace(0.001,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y.reshape(-1,1))
print(gs_results.best_params_)
print('The root mean square error is: ', np.sqrt(np.abs(gs_results.best_score_)))

{'alpha': 0.1437142857142857}
The root mean square error is:  3.868646015238579
CPU times: user 17min 30s, sys: 7min 31s, total: 25min 2s
Wall time: 14min 41s


In [92]:
%%time
model = SQRTLasso()
params = [{'alpha':np.linspace(0.001,1,num=50)}]
gs = GridSearchCV(estimator=model,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(x,y.reshape(-1,1))
print(gs_results.best_params_)
print('The mean square error is: ', np.abs(gs_results.best_score_))

{'alpha': 0.1437142857142857}
The mean square error is:  14.966421991221333
CPU times: user 17min 18s, sys: 7min 22s, total: 24min 41s
Wall time: 14min 3s


In [93]:
model = SQRTLasso(alpha=0.1437142857142857)

In [96]:
model.fit(x,y.reshape(-1,1))

In [97]:
# Compute the L2 distance
np.linalg.norm(model.coef_-beta,ord=2)

1.2019235770257295

In [128]:
true_coeffs = []
L2Distance = []
RMSE = []
model = SQRTLasso(alpha=0.1437142857142857)
for i in range(0,100):
  model.fit(X[i], Y[i].reshape(-1,1))
  true_coeffs.append(np.intersect1d(pos,np.where(model.coef_!=0)).shape[0])
  L2Distance.append(np.linalg.norm(model.coef_-beta,ord=2))
  RMSE.append(np.sqrt(MSE(Y[i],model.predict(X[i]))))

In [129]:
np.mean(true_coeffs)

27.0

In [130]:
np.mean(L2Distance)

1.3437310186892282

In [131]:
np.mean(RMSE)

3.185410399223758